#  Trabajo practico 1 IAPH Nicolas Garcia

consigna: stop-words en el Martín Fierro

Utilizar una de las bibliotecas ya mencionadas en otras tareas (PyPDF, PDFPlumber o PyMuPDF) para extraer sólo el texto completo del Martín Fierro. Ignorar imágenes, números de página, índice.

Procesar el texto para convertirlo todo a minúsculas y eliminar signos de puntuación básicos.

Separar el texto en palabras individuales (tokenización).

Identifique las stop-words usando la librería spacy. ¿Logra identificar todas las stop-words? De no ser así, indique cuáles sumaría.

## primero instalamos pymupdf y spacy ⬇




In [1]:
!pip install pymupdf spacy
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 95.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# en este paso importamos las librerias que vamos a usar mas tarde y descargamos el pdf desde la url, cuando lo abrimos le pedimos que solo nos muestre texto entonces deja de haber imagenes, las imagenes pasan a estar en formato texto y son una url que eliminaremos mediante el re, asin mismo eliminamos los numeros, numeros romanos y los "pagina", por ultimo imprimimos a ver si funciono  📚

In [2]:
import fitz
import requests
import re

# Descargar PDF
url = "https://www.argentina.gob.ar/sites/default/files/hernandez_jose_-_el_gaucho_martin_fierro.pdf"
response = requests.get(url)
response.raise_for_status()
pdf_bytes = response.content

texto_estrofas = ""

with fitz.open(stream=pdf_bytes, filetype="pdf") as pdf:
    for page in pdf:
        # Obtener texto del bloque tal cual
        page_text = page.get_text("text")

        # divide en lineas
        for line in page_text.splitlines():
            stripped = line.strip()
            if not stripped:
                continue
             #eliminamos numeros romanos que aparecen por capitulo
            if re.fullmatch(r'[IVXLCDM]+', stripped, re.IGNORECASE):
                continue
            # eliminamos numeros de pagina
            if re.fullmatch(r'\d+', stripped):
                continue
            # esto elimina el "pagina x"
            if re.search(r'(?i)página\s*\d+', stripped):
                continue
            # eliminamos URLs
            if "http" in stripped.lower() or "www." in stripped.lower():
                continue
            # conservar todo lo demás, incluso títulos o líneas cortas
            texto_estrofas += stripped + "\n"

print(texto_estrofas[:700])


Recursos de dominio público
El gaucho
Martín Fierro
José Hernández
José Hernández
El Gaucho Martín Fierro
José Hernández (1834 - 1886)
Imágenes de dominio público. Fuente:
El Gaucho Martín Fierro
Aquí me pongo a cantar
al compás de la vigüela,
que el hombre que lo desvela
una pena estrordinaria,
como la ave solitaria
con el cantar se consuela.
Pido a los santos del cielo
que ayuden mi pensamiento:
les pido en este momento
que voy a cantar mi historia
me refresquen la memoria
y aclaren mi entendimiento.
Vengan santos milagrosos,
vengan todos en mi ayuda,
que la lengua se me añuda
y se me turba la vista;
pido a mi Dios que me asista
en una ocasión tan ruda.
Yo he visto muchos cantores,
con fam


# **✏** En este paso eliminamos los signos de puntuacion y transformamos todo a minusculas mediante .lower() y ''.join(c for c in texto_minusculas if c not in string.punctuation)


In [3]:

import string
texto_minusculas = texto_estrofas.lower()
sin_puntuacion = ''.join(c for c in texto_minusculas if c not in string.punctuation)

print(sin_puntuacion[:80])

recursos de dominio público
el gaucho
martín fierro
josé hernández
josé hernánde


# Para tokenizar las palabras dividimos el texto por cada espacio que hay con .split(), despues creamos una lista de palabras unicas y vamos recorriendo el textoe n busqueda de palabras individuales, las repetidas las pasamos y solo dejamos una vez en la lista cada una.

In [4]:
# separar palabras por espacios
palabras = sin_puntuacion.split()

# palabras únicas manteniendo el orden
palabras_unicas = []
unicas = set()
for palabra in palabras:
    if palabra not in unicas:                   #busca en el texto palabras individuales y las pone en la lista,
          unicas.add(palabra)                       # si estan repetidas las pasa de largo y si no esta la agrega
          palabras_unicas.append(palabra)


print(palabras_unicas[:20])  # primeras 20 palabras únicas, en orden

['recursos', 'de', 'dominio', 'público', 'el', 'gaucho', 'martín', 'fierro', 'josé', 'hernández', '1834', '1886', 'imágenes', 'fuente', 'aquí', 'me', 'pongo', 'a', 'cantar', 'al']


# este codigo nos analiza mediante spaCy las palabras que interpreta como stopwords, primero cargamos el modelo en español (segunda linea) , despues le digo que texto quiero analizar y el doc me genera el analisis del texto que quiero analizar, despues el stopwords_spacy nos da las palabras que estamos buscando.
# como algunas palabras no aparecen entre las mas comunes lo que hice fue buscarlas individualmente y como todas aparecieron podemos asumir que tomo bien todas las stopwords y no nos faltaria ninguna

In [17]:
import spacy

nlp = spacy.load("es_core_news_sm")
texto= sin_puntuacion
doc = nlp(texto)
# lista de stopwords
stopwords_spacy = [token.text for token in doc if token.is_stop]

# ejemplo: primeras 20 stopwords encontradas
print(stopwords_spacy[:20])
print("ellos" in nlp.Defaults.stop_words)
print("bajo" in nlp.Defaults.stop_words)  #con esto lo que hago es probar si estas stopwords estan tambien en el listado, con esto asumo que todas las palabras de su mismo tipo tambien estan siendo listadas
print("también" in nlp.Defaults.stop_words)

['de', 'el', 'el', 'de', 'el', 'aquí', 'me', 'a', 'al', 'de', 'la', 'que', 'el', 'que', 'lo', 'una', 'como', 'la', 'con', 'el']
True
True
True
